In [166]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import folium
from tqdm import tqdm_notebook

In [2]:
def clean_it(path):
    with open(path, 'r') as f:
        lst = f.readlines()
        out = []
        for i in lst:
            row = i[2:-1] # deleting all elements we don't need 
            out.append(row)
        clean_out = out[7:]
        clean_out2 = []
        for j in range(len(clean_out)):
            clean_out2.append(list(map(int, clean_out[j].split())))        
    return clean_out2

In [3]:
path_dist="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw5"+"\\USA-road-d.CAL.gr"
path_time="C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw5"+"\\USA-road-t.CAL.gr"
path_cod= "C:\\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw5"+"\\USA-road-d.CAL.co"

In [5]:
data_dist=pd.DataFrame(list_dist)
data_dist.columns=["Node_a","Node_b","dist"]
data_dist.head()

,Node_a,Node_b,dist
0,1,1048577,456
1,1048577,1,456
2,2,1048578,2389
3,1048578,2,2389
4,3,1048579,358


In [71]:
data_time=pd.DataFrame(list_time)
data_time.columns=["Node_a","Node_b","dist"]
data_time.head()

,Node_a,Node_b,dist
0,1,1048577,1139
1,1048577,1,1139
2,2,1048578,5972
3,1048578,2,5972
4,3,1048579,895


In [73]:
data_cod=pd.DataFrame(list_cod)
data_cod.columns=["Node_a","latitude","longitude"]
data_cod.head()

,Node_a,latitude,longitude
0,1,-114315309,34133550
1,2,-114223946,34176221
2,3,-114307299,34148791
3,4,-114318765,34138889
4,5,-114347300,34042614


In [190]:
                       ########### Function to get the neighbouring nodes ##############

v=int(input("Please type the id of the node you want to start with: "))
metric=input("Please type the metric you prefer. The options are distance, time: ")
val=int(input("Please type the numerical value of the chosen metric: "))

# If the user choose spatial distance...

if (metric=="distance"):
    def recursive_function(v,val,the_list):
        df2 = data_dist[(data_dist.Node_a==v) & (data_dist.dist <= val) & (~data_dist.Node_b.isin(the_list))]
        df2['dist_left'] = val-df2['dist']
        the_list.extend(list(df2.Node_b))
        n = df2[['Node_b','dist_left']].shape[0]
        for i in range(n):
            recursive_function(df2.iloc[i,1],df2.iloc[i,3],the_list) #applying the search recursively
    def call_function(v,val):
        the_list=[]
        recursive_function(v,val,the_list)
        the_list=[v]+the_list
        neighbour=pd.DataFrame(the_list).reset_index(drop=True)
        neighbour.columns=['Neighbouring_nodes']
        return(neighbour) 
    
# If the user choose time...
elif(metric=="time"):
    def recursive_function(v,val,the_list):
        df2 = data_time[(data_time.Node_a==v) & (data_time.dist <= val) & (~data_time.Node_b.isin(the_list))]
        df2['dist_left'] = val-df2['dist']
        the_list.extend(list(df2.Node_b))
        n = df2[['Node_b','dist_left']].shape[0]
        for i in range(n):
            recursive_function(df2.iloc[i,1],df2.iloc[i,3],the_list) #applying the search recursively
    def call_function(v,val):
        the_list=[]
        recursive_function(v,val,the_list)
        the_list=[v]+the_list
        neighbour=pd.DataFrame(df2,the_list).reset_index(drop=True)
        neighbour.columns=['Neighbouring_nodes']
        return(neighbour) 
else: 
    print("Wrong option chosen")

# Check the code....

neighbour=call_function(v,val)
neighbour

Please type the id of the node you want to start with: 2
Please type the metric you prefer. The options are distance, time: distance
Please type the numerical value of the chosen metric: 6500


C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Neighbouring_nodes
0,2
1,1048578
2,1050174
3,2462
4,2
5,1990
6,1991
7,1050172
8,1993
9,1994


###  Visualization

In [191]:
# In order to visualize, first we construct the data set for the neighbouring nodes with all their longitudinal details 
# The details can be found from the dataset "data_cod"
neighbour.columns=["Node_a"]
neighbour=pd.merge(neighbour,data_dist,on="Node_a",how='inner')

In [192]:
neighbour=pd.merge(neighbour,data_cod,on="Node_a",how='inner')
neighbour

,Node_a,Node_b,dist,latitude,longitude
0,2,1048578,2389,-114223946,34176221
1,2,1050174,390,-114223946,34176221
2,2,2462,558,-114223946,34176221
3,2,1048578,2389,-114223946,34176221
4,2,1050174,390,-114223946,34176221
...,...,...,...,...,...
71,2001,2000,2102,-114220062,34173269
72,1989,1988,1646,-114224826,34173199
73,1989,1997,524,-114224826,34173199
74,1050774,1993,1512,-114223763,34174522


In [193]:
import decimal

In [200]:
map = folium.Map(location=[np.mean((neighbour['longitude']).tolist()),
                           np.mean((neighbour['latitude']).tolist())], default_zoom_start=6)

In [201]:
for i in tqdm_notebook(range(len(neighbour))):
    
    folium.features.RegularPolygonMarker(location = [(neighbour['longitude'].values)[0], (neighbour['latitude'].values)[0]],
                                        popup='Here you start',fill_color='green',radius = 9.5,fill_opacity = 1).add_to(map)

    folium.features.RegularPolygonMarker(location = [(neighbour['longitude'].values)[i], (neighbour['latitude'].values)[i]],
                                                 radius = 9.5, weight = 4,fill_opacity = 0.8).add_to(map)
#to add the edge between two nodes
for i in tqdm_notebook(range(len(neighbour))):
    folium.PolyLine(locations = [(((neighbour['longitude']).values)[i],((neighbour['latitude']).values)[i]), 
            (((neighbour['longitude']).values)[i], 
             ((neighbour['latitude']).values)[i])],
                    line_opacity = 0.65).add_to(map) 
#weight represent the dimension of the node
map.save("node_traversal_map.html")
 
map

In [196]:
map = folium.Map(location = [-114.223946, 34.176221], zoom_start = 6)

In [202]:
coor = []
for node in (neighbour["Node_a"]):
    
    longitude = str(data_cod.iloc[int(node)-1]['longitude'])
    longitude = decimal.Decimal(longitude[:2] + "." + longitude[2:])
    
    latitude = str(data_cod.iloc[int(node)-1]['latitude'])
    latitude = decimal.Decimal(latitude[:4] + "." + latitude[4:])
    
    folium.Marker([longitude, latitude]).add_to(map)
    coor.append((longitude, latitude))

In [203]:
folium.PolyLine(coor).add_to(map)

In [199]:
map
